In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import sys
import os
from geopy import distance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import geopandas as gpd
from shapely.ops import unary_union
import cartopy.io.shapereader as shpreader
from shapely.prepared import prep
import shapely.geometry as sgeom

In [ ]:
df = pd.read_csv('hurdat2_land.csv')

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
bth = df[df['datetime']>datetime(2000,6,24)]

In [ ]:
gom_coords = [(25.68,-97.23),
              (26.37,-97.38),
              (26.87,-97.47),
              (27.34,-97.39),
              (28.0,-96.9),
              (28.27,-96.46),
              (28.52,-96.1),
              (28.76,-95.54),
              (29.1,-95.06),
              (29.49,-94.49),
              (29.63,-94),
              (29.75,-93.59),
              (29.77,-93.17),
              (29.63,-92.7),
              (29.48,-92.08),
              (29.18,-91.18),
              (29.06,89.99),
              (29.13,-89.23),
              (29.76,-89.4),
              (30.28,-88.76),
              (30.21,-87.88),
              (30.33,-87.3),
              (30.4,-86.77),
              (30.34,-86.29),
              (30.1,-85.73),
              (29.94,-85.45),
              (26.69,-85.30),
              (29.75,-84.83),
              (29.9,-84.48),
              (30.1,-84.22),
              (30.04,-83.94),
              (29.89,-83.66),
              (29.47,-83.29),
              (29.03,-82.77),
              (28.41,-82.68),
              (27.81,-82.8),
              (27.4,-82.59),
              (27.12,-82.47),
              (26.66,-82.25),
              (26.4,-82.06),
              (26.24,-81.82),
              (25.93,-81.74),
              (25.83,-81.47),
              (25.64,-81.25),
              (25.41,-81.15),
              (25.2,-81.16),
              (24.54,-81.81),
              (24.69,-81.1),
              (24.89,-80.64),
              (25.15,-80.35),
              (25.42,-80.21)
]

In [ ]:
texas_coords = [
    (25.9,-97.13),
    (26.36,-99.05),
    (27.58,-99.54),
    (29.51,-101.2),
    (29.78,-101.6),
    (29.72,-102.66),
    (28.98,-103.16),
    (29.36,-104.1),
    (29.66,-104.52),
    (30.21,-104.7),
    (30.6,-104.94),
    (31.77,-106.51),
    (31.87,-106.63),
    (32,-106.62),
    (32,-103.06),
    (36.5,-103.05),
    (36.5,-100),
    (34.6,-100),
    (34.6,-99.97),
    (34.5,-99.95),
    (34.5,-99.9),
    (34.37,-99.66),
    (34.41,-99.58),
    (34.37,-99.4),
    (34.46,-99.37),
    (33.55,-94.05),
    (32,-94),
    (31.18,-93.5),
    (30.79,-93.57),
    (30.49,-93.71),
    (30.14,-93.7),
    (29.81,-93.925),
    (29.68,-93.84),
    (29.68,-94.028),
    (29.4,-94.7),
    (29.3,-94.74),
    (28.86,-95.38),
    (28.36,-96.37),
    (27.56,-97.22),
]

In [ ]:
storm_id =[]
for storm_number in bth['Number'].unique():
    storm = bth[bth['Number'] == storm_number]
    for index in storm.index:
        for point in texas_coords: # change the coords accordingly
            if distance.geodesic((storm['Lat'][index],storm['Lon'][index]), point).miles < 60.1: # 60 miles
                storm_id.append(storm_number)
storm_id = pd.DataFrame(storm_id, columns=['storm_id'])

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import pandas as pd
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.cm import ScalarMappable

In [ ]:
def apply_circular_mask(data, radius_km, grid_resolution_km):
    """
    Apply a circular mask to the data array.

    Parameters:
        data (ndarray): 2D precipitation data array.
        radius_km (float): Radius for the circular mask in kilometers.
        grid_resolution_km (float): Resolution of the grid in kilometers per cell.

    Returns:
        ndarray: Masked data with values outside the radius set to NaN.
    """
    center = (data.shape[0] // 2, data.shape[1] // 2)
    y, x = np.ogrid[:data.shape[0], :data.shape[1]]
    distance_from_center = np.sqrt((x - center[1]) ** 2 + (y - center[0]) ** 2)
    radius_in_cells = radius_km / grid_resolution_km
    mask = distance_from_center <= radius_in_cells
    masked_data = np.where(mask, data, np.nan)
    return masked_data

In [ ]:
def create_shared_colorbar():
    """
    Create a shared colorbar for all plots with distinct colors for each millimeter from 1 to 12.
    Values above 12 mm will be capped at 12 mm.

    Returns:
        cmap, norm: Colormap and normalization for the plots.
    """
    levels = np.arange(0, 15, 0.5)
    base_cmap = plt.get_cmap('nipy_spectral_r', len(levels) - 1)

    # Modify the base colormap to replace black with magenta
    colors = base_cmap(np.arange(base_cmap.N))
    colors[-1] = [0.35, 0, 0.52, 1]  # RGBA for magenta
    cmap = ListedColormap(colors)

    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    return cmap, norm

In [ ]:
def plot_precip_heatmap(composite_precip, category, cmap, norm):
    """
    Plot composite precipitation heatmap for a given category.

    Parameters:
        composite_precip (ndarray): Composite precipitation data.
        category (str): Storm category (e.g., HU, TD, TS).
        cmap: Colormap for the plot.
        norm: Normalization for the plot.
    """
    # Apply circular mask (assuming 10 km grid resolution)
    circular_precip = apply_circular_mask(composite_precip, radius_km=700, grid_resolution_km=10)

    # Cap precipitation values at 12 mm
    circular_precip = np.minimum(circular_precip, 12)

    # Plot the precipitation heatmap
    fig, ax = plt.subplots(figsize=(8, 8))

    ax.set_title(f"Composite Precipitation Heatmap for {category}")
    heatmap = ax.imshow(circular_precip, cmap=cmap, norm=norm, origin='lower')

    # Draw circular boundaries and add radius markings
    center = (circular_precip.shape[0] // 2, circular_precip.shape[1] // 2)
    for radius_km in range(100, 700, 100):
        radius_in_cells = radius_km / 10
        circle = plt.Circle(center[::-1], radius_in_cells, color='black', fill=False, linestyle='-', linewidth=0.5, alpha=0.8)
        ax.add_artist(circle)
        ax.text(center[1], center[0] - radius_in_cells - 5, f"{radius_km} km", color='black', fontsize=6, ha='center', va='bottom')

    # Draw a horizontal line through the center
    ax.axhline(y=center[0], color='black', linestyle='-', linewidth=0.5, alpha=0.8)

    # Draw a vertical line through the center
    ax.axvline(x=center[1], color='black', linestyle='-', linewidth=0.5, alpha=0.8)
    
    # Remove x and y axis
    ax.axis('off')

    # Add shared colorbar below the plot
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
    cbar.set_label("Precipitation (mm/hr)")

    figname = f'composite_precipitation_{category}.png' # change the name according to the territory considered
    # Save the plot
    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# Process all files in the directory and create composites
def process_directory(directory, bth, storm_ids):
    """
    Process all NetCDF files in the directory to create composite heatmaps for categories based on storm type and maximum sustained speed.

    Parameters:
        directory (str): Path to the directory containing NetCDF files.
        bth (pd.DataFrame): DataFrame containing storm metadata, including maximum sustained wind speed.
    """
    # Ensure the datetime column is in the correct format
    bth.loc[:, 'datetime'] = pd.to_datetime(bth['datetime'], format='%Y-%m-%d %H:%M:%S')
    
    # Filter storms by provided storm IDs
    bth = bth[bth['Number'].isin(storm_ids['storm_id'])] #if using whole data, comment this line

    # Classify storms based on their type and maximum sustained wind speed
    tropical_depressions = bth[bth['Status'] == 'TD']
    tropical_storms = bth[bth['Status'] == 'TS']
    high_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] > 95)]
    low_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] <= 95)]

    categories = {
        "Tropical_Depression": tropical_depressions,
        "Tropical_Storm": tropical_storms,
        "Category_3_4_5_Hurricanes": high_category_hurricanes,
        "Category_1_2_Hurricanes": low_category_hurricanes
    }

    # Create a shared colormap and normalization
    cmap, norm = create_shared_colorbar()

    for category_name, storms in categories.items():
        composite_precip = None
        count = 0

        print(f"Processing category: {category_name}")

        for _, storm in storms.iterrows():
            storm_id = storm['Number']
            datetime_str = storm['datetime'].strftime('%m_%d_%H_%M')

            # Match the corresponding NetCDF file
            nc_file_pattern = f"{storm_id}_{datetime_str}_.*\.nc"
            matching_files = [f for f in os.listdir(directory) if re.search(nc_file_pattern, f)]

            if not matching_files:
                #print(f"No matching files for storm {storm_id} at {datetime_str}")
                continue

            for file in matching_files:
                #print(f"Processing file: {file}")
                file_path = os.path.join(directory, file)
                data = xr.open_dataset(file_path)
                precip = data['precip'].values

                # Replace negative values with zero
                precip[precip < 0] = 0

                # Initialize composite
                if composite_precip is None:
                    composite_precip = precip
                else:
                    composite_precip += precip

                count += 1

        if count > 0:
            composite_precip /= count
            plot_precip_heatmap(composite_precip, category_name, cmap, norm)
        else:
            print(f"No data to plot for category: {category_name}")

In [ ]:
# without considering the inland and offshore subcategories
directory = "/scratch/09295/naveens/rotated_data_hurricane/unmasked/"  # Replace with the actual directory path
process_directory(directory, bth, storm_id)

In [ ]:
#considering the subcategories
def process_directory(directory, bth, storm_ids):
    """
    Process all NetCDF files in the directory to create composite heatmaps for categories based on storm type and maximum sustained speed.

    Parameters:
        directory (str): Path to the directory containing NetCDF files.
        bth (pd.DataFrame): DataFrame containing storm metadata, including maximum sustained wind speed.
        storm_ids (pd.DataFrame): DataFrame containing the storm IDs to process.
    """
    # Filter storms by provided storm IDs
    bth = bth[bth['Number'].isin(storm_ids['storm_id'])]

    # Classify storms based on their type and maximum sustained wind speed
    tropical_depressions = bth[bth['Status'] == 'TD']
    tropical_storms = bth[bth['Status'] == 'TS']
    high_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] > 95)]
    low_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] <= 95)]

    inland = bth[(bth['land_minus_350'] == 1) & (bth['land_plus_350'] == 1) & (bth['inland'] == 1)]
    offshore = bth[(bth['land_minus_350'] == 0) & (bth['land_plus_350'] == 0) & (bth['inland'] == 0)]
    transition = bth[(bth['land_minus_350'] == 0) & (bth['land_plus_350'] == 1) & (bth['inland'] == 0)]

    categories = {
        "Tropical_Depression": tropical_depressions,
        "Tropical_Storm": tropical_storms,
        "Category_3_4_5_Hurricanes": high_category_hurricanes,
        "Category_1_2_Hurricanes": low_category_hurricanes
    }

    subcategories = {
        "Inland": inland,
        "Offshore": offshore,
        "Transition": transition
    }

    # Create a shared colormap and normalization
    cmap, norm = create_shared_colorbar()

    for category_name, storms in categories.items():
        for subcategory_name, subcategory_storms in subcategories.items():
            composite_precip = None
            count = 0

            print(f"Processing {category_name} - {subcategory_name}")

            filtered_storms = storms.merge(subcategory_storms, how='inner', on=['Number', 'datetime', 'Status'])

            for _, storm in filtered_storms.iterrows():
                storm_id = storm['Number']
                datetime_str = storm['datetime'].strftime('%m_%d_%H_%M')

                # Match the corresponding NetCDF file
                nc_file_pattern = f"{storm_id}_{datetime_str}_.*\.nc"
                matching_files = [f for f in os.listdir(directory) if re.search(nc_file_pattern, f)]

                if not matching_files:
                    #print(f"No matching files for storm {storm_id} at {datetime_str}")
                    continue

                for file in matching_files:
                    #print(f"Processing file: {file}")
                    file_path = os.path.join(directory, file)
                    data = xr.open_dataset(file_path)
                    precip = data['precip'].values

                    # Replace negative values with zero
                    precip[precip < 0] = 0

                    # Initialize composite
                    if composite_precip is None:
                        composite_precip = precip
                    else:
                        composite_precip += precip

                    count += 1

            if count > 0:
                composite_precip /= count
                combined_category = f"{category_name}_{subcategory_name}"
                plot_precip_heatmap(composite_precip, combined_category, cmap, norm)
            else:
                print(f"No data to plot for {category_name} - {subcategory_name}")

directory = "/scratch/09295/naveens/rotated_data_hurricane/unmasked/"  # Replace with the actual directory path
process_directory(directory, bth,storm_id)

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from matplotlib.colors import BoundaryNorm
import scipy.stats as stats

In [ ]:
def plot_combined_radial_profiles(all_profiles, max_radius_km=600):
    """
    Plot radial profiles of precipitation for all categories in the same plot.

    Parameters:
        all_profiles (dict): Dictionary with categories as keys and radial profile data as values.
        max_radius_km (int): Maximum radius in kilometers for the x-axis.
    """
    max_radius_cells = max_radius_km / 10  # Convert radius from km to grid cells

    plt.figure(figsize=(12, 8))

    category_labels = {
        "Tropical_Depression": "Tropical Depression",
        "Tropical_Storm": "Tropical Storm",
        "Category_1_2_Hurricanes": "Hurricane 1 & 2",
        "Category_3_4_5_Hurricanes": "Hurricane 3, 4 & 5"
    }

    for category, profile_data in all_profiles.items():
        mean_values, lower_bound, upper_bound = profile_data
        radial_distances_km = np.arange(len(mean_values)) * 10  # Convert grid cells to km

        plt.plot(radial_distances_km, mean_values, label=category_labels.get(category, category), linewidth=1.5)
        plt.fill_between(radial_distances_km, lower_bound, upper_bound, alpha=0.2)

    plt.xlabel("Distance from Center (km)")
    plt.ylabel("Precipitation (mm/hr)")
    plt.title("Radial Precipitation Profiles for All Categories")
    plt.legend(loc="upper right")
    plt.tight_layout()

    figname = "combined_radial_profiles.png" #change the name according to the territory
    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

In [ ]:
def process_directory_for_profiles(directory, bth, storm_ids):
    """
    Process all NetCDF files in the directory to create radial profiles for multiple categories.

    Parameters:
        directory (str): Path to the directory containing NetCDF files.
        bth (pd.DataFrame): DataFrame containing storm metadata, including maximum sustained wind speed.
        storm_ids (pd.DataFrame): DataFrame containing the storm IDs to process.

    Returns:
        dict: Dictionary of radial profiles for each category.
    """
    # Filter storms by provided storm IDs
    #bth = bth[bth['Number'].isin(storm_ids['storm_id'])]

    # Classify storms based on their type and maximum sustained wind speed
    tropical_depressions = bth[bth['Status'] == 'TD']
    tropical_storms = bth[bth['Status'] == 'TS']
    high_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] > 95)]
    low_category_hurricanes = bth[(bth['Status'] == 'HU') & (bth['Maximum Sustained Wind'] <= 95)]

    categories = {
        "Tropical_Depression": tropical_depressions,
        "Tropical_Storm": tropical_storms,
        "Category_1_2_Hurricanes": low_category_hurricanes,
        "Category_3_4_5_Hurricanes": high_category_hurricanes
    }

    all_profiles = {}

    for category_name, storms in categories.items():
        all_radial_values = []
        max_radius = 600 / 10  # 600 km converted to grid cells

        for _ in range(int(max_radius)):
            all_radial_values.append([])

        print(f"Processing {category_name}")

        for _, storm in storms.iterrows():
            storm_id = storm['Number']
            datetime_str = storm['datetime'].strftime('%m_%d_%H_%M')

            # Match the corresponding NetCDF file
            nc_file_pattern = f"{storm_id}_{datetime_str}_.*\.nc"
            matching_files = [f for f in os.listdir(directory) if re.search(nc_file_pattern, f)]

            if not matching_files:
                #print(f"No matching files for storm {storm_id} at {datetime_str}")
                continue

            for file in matching_files:
                #print(f"Processing file: {file}")
                file_path = os.path.join(directory, file)
                data = xr.open_dataset(file_path)
                precip = data['precip'].values

                # Replace negative values with zero
                precip[precip < 0] = 0

                center = (precip.shape[0] // 2, precip.shape[1] // 2)
                y, x = np.ogrid[:precip.shape[0], :precip.shape[1]]
                distance_from_center = np.sqrt((x - center[1]) ** 2 + (y - center[0]) ** 2)
                #print(distance_from_center)
                for radius in range(int(max_radius)):
                    mask = (distance_from_center >= radius) & (distance_from_center < radius + 1)
                    values = precip[mask]
                    values = values[~np.isnan(values)]
                    all_radial_values[radius].extend(values)
        mean_values = [np.mean(values) if len(values) > 0 else 0 for values in all_radial_values]
        conf_intervals = [
            stats.t.interval(
                0.99,
                len(values) - 1,
                loc=np.mean(values),
                scale=stats.sem(values)
            ) if len(values) > 1 else (0, 0)
            for values in all_radial_values
        ]

        lower_bound = [ci[0] if ci[0] > 0 else 0 for ci in conf_intervals]
        upper_bound = [ci[1] if ci[1] > 0 else 0 for ci in conf_intervals]

        all_profiles[category_name] = (mean_values, lower_bound, upper_bound)

    return all_profiles

In [ ]:
directory = "/scratch/09295/naveens/rotated_data_hurricane/unmasked/"
profiles = process_directory_for_profiles(directory, bth, storm_id)
plot_combined_radial_profiles(profiles)